In [1]:
import gym
import wandb
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define Policy and REINFORCE classes

class Policy(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Policy, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma, device, optimizer_type):
        self.policy_net = Policy(state_dim, hidden_dim, action_dim).to(device)
        self.optimizer_type = optimizer_type
        if optimizer_type == 'adam':
            self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        elif optimizer_type == 'sgd':
            self.optimizer = torch.optim.SGD(self.policy_net.parameters(), lr=learning_rate)
        else:
            raise ValueError("Unsupported optimizer type")
        self.gamma = gamma
        self.device = device

    def take_action(self, state):
        state = torch.tensor(state.reshape(1, -1), dtype=torch.float).to(self.device) 
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs) 
        action = action_dist.sample()
        return action.item()  

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']

        G = 0
        self.optimizer.zero_grad()
        for i in reversed(range(len(reward_list))):
            reward = reward_list[i]
            state = torch.tensor(state_list[i].reshape(1, -1), dtype=torch.float).to(self.device)
            action = torch.tensor(action_list[i]).view(-1, 1).to(self.device)
            log_prob = torch.log(self.policy_net(state).gather(1, action))
            G = self.gamma * G + reward
            loss = -log_prob * G
            loss.backward()
        self.optimizer.step()

# Hyperparameters
learning_rate = 1e-3
gamma = 0.99
hidden_dim = 128
num_episodes = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env_name = "CartPole-v1"

# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {"name": "regret", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 1e-5, "max": 1e-2},
        "hidden_dim": {"values": [32, 64, 128, 256]},
        "optimizer_type": {"values": ["adam", "sgd"]}
    },'project':'cartpole_without_baseline_new',
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "max_iter": 100
    }
}
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config)

def train(learning_rate, hidden_dim, optimizer_type):
    env = gym.make(env_name)
    env.reset(seed=0)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = REINFORCE(state_dim, hidden_dim, action_dim, learning_rate, gamma, device, optimizer_type)
    return_list = []

    for i_episode in range(num_episodes):
        episode_return = 0
        transition_dict = {
            "states": [],
            "actions": [],
            "next_states": [],
            "rewards": [],
            "dones": []
        }
        state, _ = env.reset()
        terminated, truncated = False, False
        while not terminated and not truncated:
            action = agent.take_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            transition_dict["states"].append(state)
            transition_dict["actions"].append(action)
            transition_dict["next_states"].append(next_state)
            transition_dict["rewards"].append(reward)
            transition_dict["dones"].append(terminated)
            state = next_state
            episode_return += reward
        return_list.append(episode_return)
        agent.update(transition_dict)


    avg = []
    for i in range(num_episodes):
        if i < 100:
            avg_return = np.mean(return_list[:i+1])
        else:
            avg_return = np.mean(return_list[i-100:i+1])
        wandb.log({"avg_scores": avg_return})
        avg.append(avg_return)


    regret = 0
    for i in avg:
        if i < 475:
            regret += 475 - i

    return regret

def run_training():
    config_defaults = {
        "learning_rate": 1e-3,
        "hidden_dim": 128,
        "optimizer_type": "adam"
    }
    config = wandb.init(config=config_defaults)
    learning_rate = config.config["learning_rate"]
    hidden_dim = config.config["hidden_dim"]
    optimizer_type = config.config["optimizer_type"]
    regret = train(learning_rate, hidden_dim, optimizer_type)

    wandb.log({"regret": regret})

# Run the sweep
wandb.agent(sweep_id, function=run_training)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 76xlzgsh
Sweep URL: https://wandb.ai/rl_shobhith/cartpole_without_baseline_new/sweeps/76xlzgsh


wandb: Agent Starting Run: yp6oq89k with config:
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0016396303816840869
wandb: 	optimizer_type: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shobhith-v (rl_shobhith). Use `wandb login --relogin` to force relogin


C:\Users\Shobhith Vadlamudi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


avg_scores,▁▁▁▁▁▂▂▂▂▃▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▆▇████
regret,▁
avg_scores,122.32673
regret,122818.66113


...

wandb: Agent Starting Run: pq7p0y5v with config:
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.008817995611176228
wandb: 	optimizer_type: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


avg_scores,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
regret,▁
avg_scores,9.39604
regret,139427.41647


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r70y311q with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.006504200179722873
wandb: 	optimizer_type: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


avg_scores,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▄▄▄▅▅▆▆▆▅▅▅▅▅▄▅▅▆▆▆▇▇█
regret,▁
avg_scores,275.61386
regret,105417.20012


wandb: Agent Starting Run: d7t0y9n8 with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.002515896865860069
wandb: 	optimizer_type: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
